In [46]:
import pybaseball as pb 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from pybaseball import statcast
import glob
import pandas_profiling as pp
from sklearn.model_selection import train_test_split as tts
from sklearn.linear_model import LogisticRegression as lr, LogisticRegressionCV as lrcv

In [47]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [48]:
gbg = pb.retrosheet.season_game_logs(2008)

In [49]:
seasons = []
for x in range(2008,2020,1):
    seasons.append(x)

In [50]:
seasons.pop()

2019

In [51]:
seasons

[2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]

In [52]:
def season_pull(year):
    name = 'overall_'+str(year)
    '''Pull from retrosheet for season indicated by year 
    EX: season_pull(2008) returns a DataFrame containing 
    the entire game by game for the 2008 MLB Season'''
    df=pb.retrosheet.season_game_logs(year)
    df.to_csv('C:\\Users\\dakot\\Documents\GitHub\\baseball-game-prediction-ds\\baseball_notebooks\\Data\\'+name+'.csv' )
    return df

In [53]:
for x in seasons:
    season_pull(x)

In [68]:
df = pd.read_csv('C:\\Users\\dakot\\Documents\\GitHub\\baseball-game-prediction-ds\\baseball_notebooks\\Data\\overall_2008.csv')
cols = []
for x in df.columns:
    cols.append(x)

In [69]:
cols.pop(0)

'Unnamed: 0'

In [71]:
path = 'C:\\Users\\dakot\Documents\\GitHub\\baseball-game-prediction-ds\\baseball_notebooks\\Data' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename,usecols=cols,index_col=None)
    li.append(df)

frame = pd.concat(li, axis=0)

In [72]:
frame.shape

(26726, 161)

In [73]:
frame.head()

,date,game_num,day_of_week,visiting_team,visiting_team_league,visiting_game_num,home_team,home_team_league,home_team_game_num,visiting_score,home_score,num_outs,day_night,completion_info,forfeit_info,protest_info,park_id,attendance,time_of_game_minutes,visiting_line_score,home_line_score,visiting_abs,visiting_hits,visiting_doubles,visiting_triples,visiting_homeruns,visiting_rbi,visiting_sac_hits,visiting_sac_flies,visiting_hbp,visiting_bb,visiting_iw,visiting_k,visiting_sb,visiting_cs,visiting_gdp,visiting_ci,visiting_lob,visiting_pitchers_used,visiting_individual_er,visiting_er,visiting__wp,visiting_balks,visiting_po,visiting_assists,visiting_errors,visiting_pb,visiting_dp,visiting_tp,home_abs,home_hits,home_doubles,home_triples,home_homeruns,home_rbi,home_sac_hits,home_sac_flies,home_hbp,home_bb,home_iw,home_k,home_sb,home_cs,home_gdp,home_ci,home_lob,home_pitchers_used,home_individual_er,home_er,home_wp,home_balks,home_po,home_assists,home_errors,home_pb,home_dp,home_tp,ump_home_id,ump_home_name,ump_first_id,ump_first_name,ump_second_id,ump_second_name,ump_third_id,ump_third_name,ump_lf_id,ump_lf_name,ump_rf_id,ump_rf_name,visiting_manager_id,visiting_manager_name,home_manager_id,home_manager_name,winning_pitcher_id,winning_pitcher_name,losing_pitcher_id,losing_pitcher_name,save_pitcher_id,save_pitcher_name,game_winning_rbi_id,game_winning_rbi_name,visiting_starting_pitcher_id,visiting_starting_pitcher_name,home_starting_pitcher_id,home_starting_pitcher_name,visiting_1_id,visiting_1_name,visiting_1_pos,visiting_2_id,visiting_2_name,visiting_2_pos,visiting_2_id.1,visiting_3_name,visiting_3_pos,visiting_4_id,visiting_4_name,visiting_4_pos,visiting_5_id,visiting_5_name,visiting_5_pos,visiting_6_id,visiting_6_name,visiting_6_pos,visiting_7_id,visiting_7_name,visiting_7_pos,visiting_8_id,visiting_8_name,visiting_8_pos,visiting_9_id,visiting_9_name,visiting_9_pos,home_1_id,home_1_name,home_1_pos,home_2_id,home_2_name,home_2_pos,home_3_id,home_3_name,home_3_pos,home_4_id,home_4_name,home_4_pos,home_5_id,home_5_name,home_5_pos,home_6_id,home_6_name,home_6_pos,home_7_id,home_7_name,home_7_pos,home_8_id,home_8_name,home_8_pos,home_9_id,home_9_name,home_9_pos,misc,acquisition_info
0,20080325,0,Tue,BOS,AL,1,OAK,AL,1,6,5,60,N,NaN,NaN,NaN,TOK01,44628.0,219,0000030012,2000020001,38,11,3,0,1,6,1,0,0,3,2,7,0,0,2,0,6,6,5,5,1,0,30,15,0,0,0,0,36,7,1,0,2,5,0,0,1,7,0,10,1,0,0,0,9,5,6,6,0,0,30,12,0,0,2,0,reedr901,Rick Reed,wendh902,Hunter Wendelstedt,timmt901,Tim Timmons,nauep901,Paul Nauert,NaN,(none),NaN,(none),frant001,Terry Francona,gereb001,Bob Geren,okajh001,Hideki Okajima,streh001,Huston Street,papej001,Jonathan Papelbon,ramim002,Manny Ramirez,matsd001,Daisuke Matsuzaka,blanj001,Joe Blanton,pedrd001,Dustin Pedroia,4,youkk001,Kevin Youkilis,3,ortid001,David Ortiz,10,ramim002,Manny Ramirez,7,lowem001,Mike Lowell,5,mossb001,Brandon Moss,9,varij001,Jason Varitek,2,ellsj001,Jacoby Ellsbury,8,lugoj001,Julio Lugo,6,buckt001,Travis Buck,9,ellim001,Mark Ellis,4,bartd001,Daric Barton,3,custj001,Jack Cust,10,browe001,Emil Brown,7,crosb002,Bobby Crosby,6,hannj001,Jack Hannahan,5,suzuk001,Kurt Suzuki,2,sweer001,Ryan Sweeney,8,NaN,Y
1,20080326,0,Wed,BOS,AL,2,OAK,AL,2,1,5,51,N,NaN,NaN,NaN,TOK01,44735.0,171,000001000,01300001x,31,5,1,0,1,1,0,0,0,3,0,13,1,0,2,0,6,5,5,5,0,0,24,8,0,0,0,0,33,9,3,0,1,5,0,0,0,4,0,7,0,0,0,0,8,4,1,1,1,0,27,5,1,0,2,0,wendh902,Hunter Wendelstedt,timmt901,Tim Timmons,nauep901,Paul Nauert,reedr901,Rick Reed,NaN,(none),NaN,(none),frant001,Terry Francona,gereb001,Bob Geren,hardr001,Rich Harden,lestj001,Jon Lester,NaN,(none),denoc001,Chris Denorfia,lestj001,Jon Lester,hardr001,Rich Harden,pedrd001,Dustin Pedroia,4,youkk001,Kevin Youkilis,3,ortid001,David Ortiz,10,ramim002,Manny Ramirez,7,lowem001,Mike Lowell,5,mossb001,Brandon Moss,9,varij001,Jason Varitek,2,crisc001,Coco Crisp,8,lugoj001,Julio Lugo,6,buckt001,Travis Buck,9,ellim001,Mark Ellis,4,bartd001,Daric Barton,3,sweem002,Mike Sweeney,10,browe001,Emil Brown,7,crosb002,Bobby Crosby

In [74]:
for x in frame:
    frame['win'] = np.where(frame['home_score']>frame['visiting_score'],frame['home_team'],frame['visiting_team'])

In [75]:
frame.win.describe()

count     26726
unique       31
top         NYA
freq       1010
Name: win, dtype: object

In [76]:
frame.head()

,date,game_num,day_of_week,visiting_team,visiting_team_league,visiting_game_num,home_team,home_team_league,home_team_game_num,visiting_score,home_score,num_outs,day_night,completion_info,forfeit_info,protest_info,park_id,attendance,time_of_game_minutes,visiting_line_score,home_line_score,visiting_abs,visiting_hits,visiting_doubles,visiting_triples,visiting_homeruns,visiting_rbi,visiting_sac_hits,visiting_sac_flies,visiting_hbp,visiting_bb,visiting_iw,visiting_k,visiting_sb,visiting_cs,visiting_gdp,visiting_ci,visiting_lob,visiting_pitchers_used,visiting_individual_er,visiting_er,visiting__wp,visiting_balks,visiting_po,visiting_assists,visiting_errors,visiting_pb,visiting_dp,visiting_tp,home_abs,home_hits,home_doubles,home_triples,home_homeruns,home_rbi,home_sac_hits,home_sac_flies,home_hbp,home_bb,home_iw,home_k,home_sb,home_cs,home_gdp,home_ci,home_lob,home_pitchers_used,home_individual_er,home_er,home_wp,home_balks,home_po,home_assists,home_errors,home_pb,home_dp,home_tp,ump_home_id,ump_home_name,ump_first_id,ump_first_name,ump_second_id,ump_second_name,ump_third_id,ump_third_name,ump_lf_id,ump_lf_name,ump_rf_id,ump_rf_name,visiting_manager_id,visiting_manager_name,home_manager_id,home_manager_name,winning_pitcher_id,winning_pitcher_name,losing_pitcher_id,losing_pitcher_name,save_pitcher_id,save_pitcher_name,game_winning_rbi_id,game_winning_rbi_name,visiting_starting_pitcher_id,visiting_starting_pitcher_name,home_starting_pitcher_id,home_starting_pitcher_name,visiting_1_id,visiting_1_name,visiting_1_pos,visiting_2_id,visiting_2_name,visiting_2_pos,visiting_2_id.1,visiting_3_name,visiting_3_pos,visiting_4_id,visiting_4_name,visiting_4_pos,visiting_5_id,visiting_5_name,visiting_5_pos,visiting_6_id,visiting_6_name,visiting_6_pos,visiting_7_id,visiting_7_name,visiting_7_pos,visiting_8_id,visiting_8_name,visiting_8_pos,visiting_9_id,visiting_9_name,visiting_9_pos,home_1_id,home_1_name,home_1_pos,home_2_id,home_2_name,home_2_pos,home_3_id,home_3_name,home_3_pos,home_4_id,home_4_name,home_4_pos,home_5_id,home_5_name,home_5_pos,home_6_id,home_6_name,home_6_pos,home_7_id,home_7_name,home_7_pos,home_8_id,home_8_name,home_8_pos,home_9_id,home_9_name,home_9_pos,misc,acquisition_info,win
0,20080325,0,Tue,BOS,AL,1,OAK,AL,1,6,5,60,N,NaN,NaN,NaN,TOK01,44628.0,219,0000030012,2000020001,38,11,3,0,1,6,1,0,0,3,2,7,0,0,2,0,6,6,5,5,1,0,30,15,0,0,0,0,36,7,1,0,2,5,0,0,1,7,0,10,1,0,0,0,9,5,6,6,0,0,30,12,0,0,2,0,reedr901,Rick Reed,wendh902,Hunter Wendelstedt,timmt901,Tim Timmons,nauep901,Paul Nauert,NaN,(none),NaN,(none),frant001,Terry Francona,gereb001,Bob Geren,okajh001,Hideki Okajima,streh001,Huston Street,papej001,Jonathan Papelbon,ramim002,Manny Ramirez,matsd001,Daisuke Matsuzaka,blanj001,Joe Blanton,pedrd001,Dustin Pedroia,4,youkk001,Kevin Youkilis,3,ortid001,David Ortiz,10,ramim002,Manny Ramirez,7,lowem001,Mike Lowell,5,mossb001,Brandon Moss,9,varij001,Jason Varitek,2,ellsj001,Jacoby Ellsbury,8,lugoj001,Julio Lugo,6,buckt001,Travis Buck,9,ellim001,Mark Ellis,4,bartd001,Daric Barton,3,custj001,Jack Cust,10,browe001,Emil Brown,7,crosb002,Bobby Crosby,6,hannj001,Jack Hannahan,5,suzuk001,Kurt Suzuki,2,sweer001,Ryan Sweeney,8,NaN,Y,BOS
1,20080326,0,Wed,BOS,AL,2,OAK,AL,2,1,5,51,N,NaN,NaN,NaN,TOK01,44735.0,171,000001000,01300001x,31,5,1,0,1,1,0,0,0,3,0,13,1,0,2,0,6,5,5,5,0,0,24,8,0,0,0,0,33,9,3,0,1,5,0,0,0,4,0,7,0,0,0,0,8,4,1,1,1,0,27,5,1,0,2,0,wendh902,Hunter Wendelstedt,timmt901,Tim Timmons,nauep901,Paul Nauert,reedr901,Rick Reed,NaN,(none),NaN,(none),frant001,Terry Francona,gereb001,Bob Geren,hardr001,Rich Harden,lestj001,Jon Lester,NaN,(none),denoc001,Chris Denorfia,lestj001,Jon Lester,hardr001,Rich Harden,pedrd001,Dustin Pedroia,4,youkk001,Kevin Youkilis,3,ortid001,David Ortiz,10,ramim002,Manny Ramirez,7,lowem001,Mike Lowell,5,mossb001,Brandon Moss,9,varij001,Jason Varitek,2,crisc001,Coco Crisp,8,lugoj001,Julio Lugo,6,buckt001,Travis Buck,9,ellim001,Mark Ellis,4,bartd001,Daric Barton,3,sweem002,Mike Sweeney,10,browe001,Emil Brown,7,crosb002,Bobb

In [77]:
for x in frame.columns:
    print(x)

date
game_num
day_of_week
visiting_team
visiting_team_league
visiting_game_num
home_team
home_team_league
home_team_game_num
visiting_score
home_score
num_outs
day_night
completion_info
forfeit_info
protest_info
park_id
attendance
time_of_game_minutes
visiting_line_score
home_line_score
visiting_abs
visiting_hits
visiting_doubles
visiting_triples
visiting_homeruns
visiting_rbi
visiting_sac_hits
visiting_sac_flies
visiting_hbp
visiting_bb
visiting_iw
visiting_k
visiting_sb
visiting_cs
visiting_gdp
visiting_ci
visiting_lob
visiting_pitchers_used
visiting_individual_er
visiting_er
visiting__wp
visiting_balks
visiting_po
visiting_assists
visiting_errors
visiting_pb
visiting_dp
visiting_tp
home_abs
home_hits
home_doubles
home_triples
home_homeruns
home_rbi
home_sac_hits
home_sac_flies
home_hbp
home_bb
home_iw
home_k
home_sb
home_cs
home_gdp
home_ci
home_lob
home_pitchers_used
home_individual_er
home_er
home_wp
home_balks
home_po
home_assists
home_errors
home_pb
home_dp
home_tp
ump_home_id
u

In [78]:
#isolate all columns with future information, these details wouldnt be known until the conclusion of the game 
drops = ['visiting_score',
'home_score',
'num_outs',
'completion_info',
'forfeit_info',
'protest_info',
'attendance',
'time_of_game_minutes',
'visiting_line_score',
'home_line_score',
'visiting_abs',
'visiting_hits',
'visiting_doubles',
'visiting_triples',
'visiting_homeruns',
'visiting_rbi',
'visiting_sac_hits',
'visiting_sac_flies',
'visiting_hbp',
'visiting_bb',
'visiting_iw',
'visiting_k',
'visiting_sb',
'visiting_cs',
'visiting_gdp',
'visiting_ci',
'visiting_lob',
'visiting_pitchers_used',
'visiting_individual_er',
'visiting_er',
'visiting__wp',
'visiting_balks',
'visiting_po',
'visiting_assists',
'visiting_errors',
'visiting_pb',
'visiting_dp',
'visiting_tp',
'home_abs',
'home_hits',
'home_doubles',
'home_triples',
'home_homeruns',
'home_rbi',
'home_sac_hits',
'home_sac_flies',
'home_hbp',
'home_bb',
'home_iw',
'home_k',
'home_sb',
'home_cs',
'home_gdp',
'home_ci',
'home_lob',
'home_pitchers_used',
'home_individual_er',
'home_er',
'home_wp',
'home_balks',
'home_po',
'home_assists',
'home_errors',
'home_pb',
'home_dp',
'home_tp',
'winning_pitcher_id',
'winning_pitcher_name',
'losing_pitcher_id',
'losing_pitcher_name',
'save_pitcher_id',
'save_pitcher_name',
'game_winning_rbi_id',
'game_winning_rbi_name',
'misc',
'acquisition_info',
'visiting_manager_id',
'home_manager_id',
'visiting_starting_pitcher_id',
'home_starting_pitcher_id',
'visiting_1_id',
'visiting_2_id',
'visiting_2_id.1',
'visiting_4_id',
'visiting_5_id',
'visiting_6_id',
'visiting_7_id',
'visiting_8_id',
'visiting_9_id',
'home_1_id',
'home_2_id',
'home_3_id',
'home_4_id',
'home_5_id',
'home_6_id',
'home_7_id',
'home_8_id',
'home_9_id',
'ump_home_id',
'ump_home_name',
'ump_first_id',
'ump_first_name',
'ump_second_id',
'ump_second_name',
'ump_third_id',
'ump_third_name' ,
'ump_lf_id',
'ump_lf_name',
'ump_rf_id',
'ump_rf_name']

In [79]:
kill_time_trvl = frame.drop(drops, axis=1)

In [80]:
kill_time_trvl.head()

,date,game_num,day_of_week,visiting_team,visiting_team_league,visiting_game_num,home_team,home_team_league,home_team_game_num,day_night,park_id,visiting_manager_name,home_manager_name,visiting_starting_pitcher_name,home_starting_pitcher_name,visiting_1_name,visiting_1_pos,visiting_2_name,visiting_2_pos,visiting_3_name,visiting_3_pos,visiting_4_name,visiting_4_pos,visiting_5_name,visiting_5_pos,visiting_6_name,visiting_6_pos,visiting_7_name,visiting_7_pos,visiting_8_name,visiting_8_pos,visiting_9_name,visiting_9_pos,home_1_name,home_1_pos,home_2_name,home_2_pos,home_3_name,home_3_pos,home_4_name,home_4_pos,home_5_name,home_5_pos,home_6_name,home_6_pos,home_7_name,home_7_pos,home_8_name,home_8_pos,home_9_name,home_9_pos,win
0,20080325,0,Tue,BOS,AL,1,OAK,AL,1,N,TOK01,Terry Francona,Bob Geren,Daisuke Matsuzaka,Joe Blanton,Dustin Pedroia,4,Kevin Youkilis,3,David Ortiz,10,Manny Ramirez,7,Mike Lowell,5,Brandon Moss,9,Jason Varitek,2,Jacoby Ellsbury,8,Julio Lugo,6,Travis Buck,9,Mark Ellis,4,Daric Barton,3,Jack Cust,10,Emil Brown,7,Bobby Crosby,6,Jack Hannahan,5,Kurt Suzuki,2,Ryan Sweeney,8,BOS
1,20080326,0,Wed,BOS,AL,2,OAK,AL,2,N,TOK01,Terry Francona,Bob Geren,Jon Lester,Rich Harden,Dustin Pedroia,4,Kevin Youkilis,3,David Ortiz,10,Manny Ramirez,7,Mike Lowell,5,Brandon Moss,9,Jason Varitek,2,Coco Crisp,8,Julio Lugo,6,Travis Buck,9,Mark Ellis,4,Daric Barton,3,Mike Sweeney,10,Emil Brown,7,Bobby Crosby,6,Jack Hannahan,5,Kurt Suzuki,2,Chris Denorfia,8,OAK
2,20080330,0,Sun,ATL,NL,1,WAS,NL,1,N,WAS11,Bobby Cox,Manny Acta,Tim Hudson,Odalis Perez,Kelly Johnson,4,Yunel Escobar,6,Chipper Jones,5,Mark Teixeira,3,Jeff Francoeur,9,Brian McCann,2,Matt Diaz,7,Mark Kotsay,8,Tim Hudson,1,Cristian Guzman,6,Lastings Milledge,8,Ryan Zimmerman,5,Nick Johnson,3,Austin Kearns,9,Paul Lo Duca,2,Elijah Dukes,7,Ronnie Belliard,4,Odalis Perez,1,WAS
3,20080331,0,Mon,PIT,NL,1,ATL,NL,2,N,ATL02,John Russell,Bobby Cox,Ian Snell,Tom Glavine,Nate McLouth,8,Freddy Sanchez,4,Jason Bay,7,Adam LaRoche,3,Xavier Nady,9,Ryan Doumit,2,Jose Bautista,5,Jack Wilson,6,Ian Snell,1,Kelly Johnson,4,Yunel Escobar,6,Chipper Jones,5,Mark Teixeira,3,Brian McCann,2,Jeff Francoeur,9,Mark Kotsay,8,Matt Diaz,7,Tom Glavine,1,PIT
4,20080331,0,Mon,MIL,NL,1,CHN,NL,1,D,CHI11,Ned Yost,Lou Piniella,Ben Sheets,Carlos Zambrano,Rickie Weeks,4,Tony Gwynn,8,Prince Fielder,3,Ryan Braun,7,Bill Hall,5,Corey Hart,9,J.J. Hardy,6,Ben Sheets,1,Jason Kendall,2,Ryan Theriot,6,Alfonso Soriano,7,Derrek Lee,3,Aramis Ramirez,5,Kosuke Fukudome,9,Mark DeRosa,4,Geovany Soto,2,Felix Pie,8,Carlos Zambrano,1,MIL


In [81]:
kill_time_trvl.describe(include='object')

,day_of_week,visiting_team,visiting_team_league,home_team,home_team_league,day_night,park_id,visiting_manager_name,home_manager_name,visiting_starting_pitcher_name,home_starting_pitcher_name,visiting_1_name,visiting_2_name,visiting_3_name,visiting_4_name,visiting_5_name,visiting_6_name,visiting_7_name,visiting_8_name,visiting_9_name,home_1_name,home_2_name,home_3_name,home_4_name,home_5_name,home_6_name,home_7_name,home_8_name,home_9_name,win
count,26726,26726,26726,26726,26726,26726,26726,26726,26726,26726,26726,26726,26726,26726,26726,26726,26726,26726,26726,26726,26726,26726,26726,26726,26726,26726,26726,26726,26726,26726
unique,7,31,2,31,2,2,42,97,99,961,953,612,881,596,598,921,1167,1334,1488,1790,602,891,581,588,896,1155,1328,1430,1381,31
top,Sat,MIL,NL,TEX,NL,N,MIL06,Joe Maddon,Joe Maddon,Justin Verlander,James Shields,Denard Span,Dustin Pedroia,Joey Votto,Adrian Beltre,Jay Bruce,Yadier Molina,Mark Reynolds,Jeff Mathis,Cliff Pennington,Denard Span,Dustin Pedroia,Ryan Braun,Adrian Beltre,Jay Bruce,Yadier Molina,J.J. Hardy,Jordy Mercer,Brendan Ryan,NYA
freq,4337,892,13766,892,13766,17947,896,892,892,179,182,585,408,639,523,319,235,154,213,196,572,416,622,522,299,256,157,225,200,1010


In [82]:
kill_time_trvl.isna().sum()

date                              0
game_num                          0
day_of_week                       0
visiting_team                     0
visiting_team_league              0
visiting_game_num                 0
home_team                         0
home_team_league                  0
home_team_game_num                0
day_night                         0
park_id                           0
visiting_manager_name             0
home_manager_name                 0
visiting_starting_pitcher_name    0
home_starting_pitcher_name        0
visiting_1_name                   0
visiting_1_pos                    0
visiting_2_name                   0
visiting_2_pos                    0
visiting_3_name                   0
visiting_3_pos                    0
visiting_4_name                   0
visiting_4_pos                    0
visiting_5_name                   0
visiting_5_pos                    0
visiting_6_name                   0
visiting_6_pos                    0
visiting_7_name             

In [105]:
kill_time_trvl.to_csv('C:\\Users\\dakot\\Documents\\GitHub\\baseball-game-prediction-ds\\baseball_notebooks\\Data\\kill_time_trvl.csv')

In [87]:
y = kill_time_trvl['win']
X = kill_time_trvl.drop(columns='win', axis = 1)

In [88]:
y.shape


(26726,)

In [89]:
# Need to encode X to model logistic, multiclass 
# Look into wrapping multiple paired team models in a single function 
# Deploy the thing!

In [90]:
X.columns

Index(['date', 'game_num', 'day_of_week', 'visiting_team', 'visiting_team_league', 'visiting_game_num', 'home_team', 'home_team_league', 'home_team_game_num', 'day_night', 'park_id', 'visiting_manager_name', 'home_manager_name', 'visiting_starting_pitcher_name', 'home_starting_pitcher_name', 'visiting_1_name', 'visiting_1_pos', 'visiting_2_name', 'visiting_2_pos', 'visiting_3_name', 'visiting_3_pos', 'visiting_4_name', 'visiting_4_pos', 'visiting_5_name', 'visiting_5_pos', 'visiting_6_name', 'visiting_6_pos', 'visiting_7_name', 'visiting_7_pos', 'visiting_8_name', 'visiting_8_pos', 'visiting_9_name', 'visiting_9_pos', 'home_1_name', 'home_1_pos', 'home_2_name', 'home_2_pos', 'home_3_name', 'home_3_pos', 'home_4_name', 'home_4_pos', 'home_5_name', 'home_5_pos', 'home_6_name', 'home_6_pos', 'home_7_name', 'home_7_pos', 'home_8_name', 'home_8_pos', 'home_9_name', 'home_9_pos'], dtype='object')

In [91]:
# use Binary encoding to take care of categoricals and labels without adding too much dimensionality 
import category_encoders as ce

In [92]:
# Categorical boolean mask
cat_mask = X.dtypes==object
# filter categorical columns using mask and turn it into a list
cat_cols = X.columns[cat_mask].tolist()

In [93]:
ids = ['visiting_manager_id',
'home_manager_id',
'visiting_starting_pitcher_id',
'home_starting_pitcher_id',
'visiting_1_id',
'visiting_2_id',
'visiting_2_id.1',
'visiting_4_id',
'visiting_5_id',
'visiting_6_id',
'visiting_7_id',
'visiting_8_id',
'visiting_9_id',
'home_1_id',
'home_2_id',
'home_3_id',
'home_4_id',
'home_5_id',
'home_6_id',
'home_7_id',
'home_8_id',
'home_9_id']

In [94]:
for x in cat_cols:
    for i in ids:
        if x==i:
            cat_cols.remove(x)

In [95]:
cat_cols


['day_of_week',
 'visiting_team',
 'visiting_team_league',
 'home_team',
 'home_team_league',
 'day_night',
 'park_id',
 'visiting_manager_name',
 'home_manager_name',
 'visiting_starting_pitcher_name',
 'home_starting_pitcher_name',
 'visiting_1_name',
 'visiting_2_name',
 'visiting_3_name',
 'visiting_4_name',
 'visiting_5_name',
 'visiting_6_name',
 'visiting_7_name',
 'visiting_8_name',
 'visiting_9_name',
 'home_1_name',
 'home_2_name',
 'home_3_name',
 'home_4_name',
 'home_5_name',
 'home_6_name',
 'home_7_name',
 'home_8_name',
 'home_9_name']

In [96]:
encoder = ce.BinaryEncoder(cols=cat_cols)
X_cat_enc = encoder.fit_transform(X)

In [97]:
X_cat_enc.shape #soooo many columns, but we shall workwith it for now

(26726, 295)

In [98]:
X_cat_enc.shape, y.shape

((26726, 295), (26726,))

In [99]:
X_train, X_test, y_train, y_test = tts(X_cat_enc,y,test_size=0.2, random_state=42 )

In [100]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((21380, 295), (5346, 295), (21380,), (5346,))

In [101]:
# l_model = lr(multi_class='multinomial',solver='newton-cg', max_iter=300)
# l_model.fit(X_train,y_train)

In [102]:
# drag df back in for other tests and less overhead 
# will need to split again 
# try catboost without encoding

In [103]:
import pybaseball as pb 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from pybaseball import statcast
import glob
import pandas_profiling as pp
from sklearn.model_selection import train_test_split as tts
from sklearn.linear_model import LogisticRegression as lr, LogisticRegressionCV as lrcv
import json

In [106]:
reup = pd.read_csv('C:\\Users\\dakot\\Documents\\GitHub\\baseball-game-prediction-ds\\baseball_notebooks\\Data\\kill_time_trvl.csv')
reup.shape

(26726, 53)

In [108]:
reup.drop('Unnamed: 0', axis=1,inplace=True)

### Given the format we will be recieiving from the web request we need to change some team abbreviations so the model is prepared for them

### Current team abbreviations are ANA, ARI, ATL, BAL, BOS, CHA (Chicago American – Whit Sox), CHN (Chicago National – Cubs), CIN, CLE, COL, DET, FLO, HOU, KCA, LAN, MIL, MIN, NYA, NYN, OAK, PHI, PIT, SDN, SEA, SFN, SLN, TBA, TEX, TOR, and WAS

### need to filter and change them to match [ARI,ATL,BAL,BOS,CHC,CIN,CLE,COL,CWS,DET,HOU,KC,LAA,LAD,MIW,MIL,MIN,NYM,NYY,OAK,PHI,PIT,SD,SEA,SF,STL,TB,TEX,TOR,WSH

In [109]:
#dict and replace method will need to run model and pickle again. 
change_name = {'ANA':'LAA', 'ARI':'ARI', 'ATL':'ATL', 'BAL':'BAL', 'BOS':'BOS', 'CHA':'CWS', 'CHN':'CHC', 'CIN':'CIN', 'CLE':'CLE', 'COL':'COL', 'DET':'DET', 'FLO':'MIA', 'HOU':'HOU', 'KCA':'KC', 'LAN':'LAD', 'MIL':'MIL', 'MIN':'MIN', 'NYA':'NYY', 'NYN':'NYM', 'OAK':'OAK', 'PHI':'PHI', 'PIT':'PIT', 'SDN':'SD', 'SEA':'SEA', 'SFN':'SF', 'SLN':'STL', 'TBA':'TB', 'TEX':'TEX', 'TOR':'TOR', 'WAS':'WSH'}

In [110]:
reup= reup.replace(change_name)

In [118]:
reup.home_team.unique()

## Now we need to handle discrepancies in how the venues or parks are identified, have to map for that lets go Numeric. 


In [120]:
parks = reup.park_id.unique()

In [123]:
plist = list(parks)

In [142]:
park_rep = {'TOK01': 147, 'WAS11': 121, 'ATL02':136, 'CHI11': 133, 'CIN09': 116, 'MIA01':118, 'Los03': 113, 'PHI13': 110, 'SAN02': 124, 'BAL12': 123, 'CLE08': 126, 'DET05': 111, 'MIN03': 129, 'SEA03': 128, 'STL10': 107, 'NYC16': 134, 'OAK01': 64, 'DEN02': 112, 'MIL06': 119, 'ANA01': 105, 'TOR02': 127, 'PHO01': 108, 'HOU03': 120, 'PIT08': 125, 'SFO03': 106, 'CHI12': 132, 'NYC17': 109, 'BOS07': 114, 'KAN06': 117, 'STP01': 130, 'ARL02': 115, 'NYC20': 109, 'NYC21': 134, 'MIN04': 129, 'MIA02': 118, 'ATL03': 136}

In [146]:
reup = reup.replace(park_rep)


### Organize days of week to numeric

In [149]:
reup.day_of_week.unique()

array(['Tue', 'Wed', 'Sun', 'Mon', 'Thu', 'Fri', 'Sat'], dtype=object)

In [150]:
day_dict = {'Sun': 0, 'Mon': 1, 'Tue': 2, 'Wed': 3, 'Thu': 4, 'Fri': 5, 'Sat': 6}

In [151]:
reup = reup.replace(day_dict)

In [152]:
reup.head()

,date,game_num,day_of_week,visiting_team,visiting_team_league,visiting_game_num,home_team,home_team_league,home_team_game_num,day_night,park_id,visiting_manager_name,home_manager_name,visiting_starting_pitcher_name,home_starting_pitcher_name,visiting_1_name,visiting_1_pos,visiting_2_name,visiting_2_pos,visiting_3_name,visiting_3_pos,visiting_4_name,visiting_4_pos,visiting_5_name,visiting_5_pos,visiting_6_name,visiting_6_pos,visiting_7_name,visiting_7_pos,visiting_8_name,visiting_8_pos,visiting_9_name,visiting_9_pos,home_1_name,home_1_pos,home_2_name,home_2_pos,home_3_name,home_3_pos,home_4_name,home_4_pos,home_5_name,home_5_pos,home_6_name,home_6_pos,home_7_name,home_7_pos,home_8_name,home_8_pos,home_9_name,home_9_pos,win
0,20080325,0,2,BOS,AL,1,OAK,AL,1,N,147,Terry Francona,Bob Geren,Daisuke Matsuzaka,Joe Blanton,Dustin Pedroia,4,Kevin Youkilis,3,David Ortiz,10,Manny Ramirez,7,Mike Lowell,5,Brandon Moss,9,Jason Varitek,2,Jacoby Ellsbury,8,Julio Lugo,6,Travis Buck,9,Mark Ellis,4,Daric Barton,3,Jack Cust,10,Emil Brown,7,Bobby Crosby,6,Jack Hannahan,5,Kurt Suzuki,2,Ryan Sweeney,8,BOS
1,20080326,0,3,BOS,AL,2,OAK,AL,2,N,147,Terry Francona,Bob Geren,Jon Lester,Rich Harden,Dustin Pedroia,4,Kevin Youkilis,3,David Ortiz,10,Manny Ramirez,7,Mike Lowell,5,Brandon Moss,9,Jason Varitek,2,Coco Crisp,8,Julio Lugo,6,Travis Buck,9,Mark Ellis,4,Daric Barton,3,Mike Sweeney,10,Emil Brown,7,Bobby Crosby,6,Jack Hannahan,5,Kurt Suzuki,2,Chris Denorfia,8,OAK
2,20080330,0,0,ATL,NL,1,WSH,NL,1,N,121,Bobby Cox,Manny Acta,Tim Hudson,Odalis Perez,Kelly Johnson,4,Yunel Escobar,6,Chipper Jones,5,Mark Teixeira,3,Jeff Francoeur,9,Brian McCann,2,Matt Diaz,7,Mark Kotsay,8,Tim Hudson,1,Cristian Guzman,6,Lastings Milledge,8,Ryan Zimmerman,5,Nick Johnson,3,Austin Kearns,9,Paul Lo Duca,2,Elijah Dukes,7,Ronnie Belliard,4,Odalis Perez,1,WSH
3,20080331,0,1,PIT,NL,1,ATL,NL,2,N,136,John Russell,Bobby Cox,Ian Snell,Tom Glavine,Nate McLouth,8,Freddy Sanchez,4,Jason Bay,7,Adam LaRoche,3,Xavier Nady,9,Ryan Doumit,2,Jose Bautista,5,Jack Wilson,6,Ian Snell,1,Kelly Johnson,4,Yunel Escobar,6,Chipper Jones,5,Mark Teixeira,3,Brian McCann,2,Jeff Francoeur,9,Mark Kotsay,8,Matt Diaz,7,Tom Glavine,1,PIT
4,20080331,0,1,MIL,NL,1,CHC,NL,1,D,133,Ned Yost,Lou Piniella,Ben Sheets,Carlos Zambrano,Rickie Weeks,4,Tony Gwynn,8,Prince Fielder,3,Ryan Braun,7,Bill Hall,5,Corey Hart,9,J.J. Hardy,6,Ben Sheets,1,Jason Kendall,2,Ryan Theriot,6,Alfonso Soriano,7,Derrek Lee,3,Aramis Ramirez,5,Kosuke Fukudome,9,Mark DeRosa,4,Geovany Soto,2,Felix Pie,8,Carlos Zambrano,1,MIL


In [153]:
X_2 = reup.drop(['win'],axis=1)
y_2 = reup['win']

In [154]:
X_tr, X_tst, y_tr, y_tst = tts(X_2,y_2,test_size=0.2, random_state=42 )

In [155]:
from sklearn.metrics import accuracy_score as score

In [156]:
from catboost import CatBoostClassifier as cb


In [157]:
X_2.dtypes

date                               int64
game_num                           int64
day_of_week                        int64
visiting_team                     object
visiting_team_league              object
visiting_game_num                  int64
home_team                         object
home_team_league                  object
home_team_game_num                 int64
day_night                         object
park_id                           object
visiting_manager_name             object
home_manager_name                 object
visiting_starting_pitcher_name    object
home_starting_pitcher_name        object
visiting_1_name                   object
visiting_1_pos                     int64
visiting_2_name                   object
visiting_2_pos                     int64
visiting_3_name                   object
visiting_3_pos                     int64
visiting_4_name                   object
visiting_4_pos                     int64
visiting_5_name                   object
visiting_5_pos  

In [158]:
X_2.home_team

0        OAK
1        OAK
2        WSH
3        ATL
4        CHC
5        CIN
6        MIA
7        LAD
8        PHI
9         SD
10       BAL
11       CLE
12       DET
13       MIN
14       SEA
15       MIA
16       LAD
17        SD
18       STL
19       MIN
20       NYY
21       OAK
22       SEA
23       ATL
24       CHC
25       CIN
26       MIA
27       LAD
28       PHI
29        SD
30       STL
31       BAL
32       CLE
33       DET
34       MIN
35       NYY
36       OAK
37       SEA
38       ATL
39       CHC
40       CIN
41       PHI
42        SD
43       STL
44       CLE
45       DET
46       MIN
47       NYY
48       CHC
49       CIN
50       COL
51       MIA
52       MIL
53        SD
54       STL
55       LAA
56       BAL
57       DET
58       MIN
59       NYY
60       OAK
61       TOR
62       ATL
63       CHC
64       CIN
65       COL
66       MIA
67       MIL
68        SD
69       STL
70       LAA
71       BAL
72       DET
73       MIN
74       NYY
75       OAK
76       TOR

In [160]:
categorical_features_indices = np.where(X_2.dtypes != np.int64)[0]

In [161]:
categorical_features_indices

array([ 3,  4,  6,  7,  9, 10, 11, 12, 13, 14, 15, 17, 19, 21, 23, 25, 27,
       29, 31, 33, 35, 37, 39, 41, 43, 45, 47, 49], dtype=int64)

In [163]:
kitty = cb(
    iterations=40,
    depth=10,
    learning_rate=0.01,
    #random_seed=42,
    loss_function='MultiClass',
    verbose=True)



In [164]:
kitty.fit(X_tr,y_tr,cat_features=categorical_features_indices)

0:	learn: 3.3568087	total: 16s	remaining: 10m 24s
1:	learn: 3.3207971	total: 30s	remaining: 9m 29s
2:	learn: 3.2882670	total: 44.4s	remaining: 9m 7s
3:	learn: 3.2503369	total: 58.7s	remaining: 8m 48s
4:	learn: 3.2120794	total: 1m 12s	remaining: 8m 30s
5:	learn: 3.1771695	total: 1m 27s	remaining: 8m 14s
6:	learn: 3.1410762	total: 1m 41s	remaining: 7m 57s
7:	learn: 3.1107026	total: 1m 55s	remaining: 7m 43s
8:	learn: 3.0755187	total: 2m 9s	remaining: 7m 26s
9:	learn: 3.0443762	total: 2m 24s	remaining: 7m 12s
10:	learn: 3.0171736	total: 2m 38s	remaining: 6m 57s
11:	learn: 2.9865979	total: 2m 52s	remaining: 6m 42s
12:	learn: 2.9600192	total: 3m 9s	remaining: 6m 34s
13:	learn: 2.9333728	total: 3m 24s	remaining: 6m 19s
14:	learn: 2.9105395	total: 3m 38s	remaining: 6m 4s
15:	learn: 2.8884453	total: 3m 52s	remaining: 5m 49s
16:	learn: 2.8683752	total: 4m 7s	remaining: 5m 34s
17:	learn: 2.8432322	total: 4m 21s	remaining: 5m 19s
18:	learn: 2.8211251	total: 4m 35s	remaining: 5m 4s
19:	learn: 2.797

In [165]:
pred_class = kitty.predict(X_tst)
pred_probs = kitty.predict_proba(X_tst)

In [166]:
cat_score = score(y_tst,pred_class)
print('Accuracy of catboost=',cat_score)

Accuracy of catboost= 0.5342312008978676


In [167]:
imps = kitty.get_feature_importance(prettified=True)
scrs = kitty.score(X_tst,y_tst)
probs = kitty.predict_proba(data=(X_tst[:1]))

In [168]:
case = X_2.iloc[1000].to_json()

In [169]:
type(case)
data = json.loads(case)
game =list(data.values())

In [170]:
game

[20080612,
 0,
 4,
 'MIN',
 'AL',
 67,
 'CLE',
 'AL',
 67,
 'N',
 126,
 'Ron Gardenhire',
 'Eric Wedge',
 'Livan Hernandez',
 'Aaron Laffey',
 'Carlos Gomez',
 8,
 'Alexi Casilla',
 4,
 'Joe Mauer',
 2,
 'Justin Morneau',
 3,
 'Michael Cuddyer',
 9,
 'Jason Kubel',
 7,
 'Craig Monroe',
 10,
 'Brendan Harris',
 6,
 'Matt Macri',
 5,
 'Grady Sizemore',
 8,
 'Jamey Carroll',
 4,
 'Ben Francisco',
 7,
 'Ryan Garko',
 3,
 'David Dellucci',
 10,
 'Jhonny Peralta',
 6,
 'Shin-Soo Choo',
 9,
 'Casey Blake',
 5,
 'Kelly Shoppach',
 2]

In [171]:
prediction = kitty.predict(game)[0]

out=prediction[0]
wins = {'Win!': str(out)}
# type(case)

In [172]:
wins

{'Win!': 'CLE'}

In [173]:
# Pickle the darn thing! 🥒
import pickle 
model_name = 'cat_pickle.pkl'

In [174]:
loaded_model = pickle.load(open(model_name, 'rb'))
# result = loaded_model.score(X_tst, y_tst)
# print(result)

FileNotFoundError: [Errno 2] No such file or directory: 'cat_pickle.pkl'

In [175]:
case

'{"date":20080612,"game_num":0,"day_of_week":4,"visiting_team":"MIN","visiting_team_league":"AL","visiting_game_num":67,"home_team":"CLE","home_team_league":"AL","home_team_game_num":67,"day_night":"N","park_id":126,"visiting_manager_name":"Ron Gardenhire","home_manager_name":"Eric Wedge","visiting_starting_pitcher_name":"Livan Hernandez","home_starting_pitcher_name":"Aaron Laffey","visiting_1_name":"Carlos Gomez","visiting_1_pos":8,"visiting_2_name":"Alexi Casilla","visiting_2_pos":4,"visiting_3_name":"Joe Mauer","visiting_3_pos":2,"visiting_4_name":"Justin Morneau","visiting_4_pos":3,"visiting_5_name":"Michael Cuddyer","visiting_5_pos":9,"visiting_6_name":"Jason Kubel","visiting_6_pos":7,"visiting_7_name":"Craig Monroe","visiting_7_pos":10,"visiting_8_name":"Brendan Harris","visiting_8_pos":6,"visiting_9_name":"Matt Macri","visiting_9_pos":5,"home_1_name":"Grady Sizemore","home_1_pos":8,"home_2_name":"Jamey Carroll","home_2_pos":4,"home_3_name":"Ben Francisco","home_3_pos":7,"home_4_

In [6]:
import json
import requests

In [7]:
url =" http://127.0.0.1:5000/"
dats = {"date": 20080612,"game_num": 0,"day_of_week": "Thu",
 "visiting_team": "MIN", "visiting_team_league": "AL",
 "visiting_game_num": 67, "home_team": "CLE",
 "home_team_league": "AL", "home_team_game_num": 67,
 "day_night": "N", "park_id": "CLE08",
 "visiting_manager_name": "Ron Gardenhire",
 "home_manager_name": "Eric Wedge",
 "visiting_starting_pitcher_name": "Livan Hernandez",
 "home_starting_pitcher_name": "Aaron Laffey",
 "visiting_1_name": "Carlos Gomez",
 "visiting_1_pos": 8, "visiting_2_name": "Alexi Casilla",
 "visiting_2_pos": 4, "visiting_3_name": "Joe Mauer",
 "visiting_3_pos": 2, "visiting_4_name": "Justin Morneau",
 "visiting_4_pos": 3, "visiting_5_name": "Michael Cuddyer",
 "visiting_5_pos": 9, "visiting_6_name": "Jason Kubel",
 "visiting_6_pos": 7, "visiting_7_name": "Craig Monroe",
 "visiting_7_pos": 10, "visiting_8_name": "Brendan Harris",
 "visiting_8_pos": 6, "visiting_9_name": "Matt Macri",
 "visiting_9_pos": 5, "home_1_name": "Grady Sizemore",
 "home_1_pos": 8, "home_2_name": "Jamey Carroll",
 "home_2_pos": 4, "home_3_name": "Ben Francisco",
 "home_3_pos": 7, "home_4_name": "Ryan Garko",
 "home_4_pos": 3, "home_5_name": "David Dellucci",
 "home_5_pos": 10, "home_6_name": "Jhonny Peralta",
 "home_6_pos": 6, "home_7_name": "Shin-Soo Choo",
 "home_7_pos": 9, "home_8_name": "Casey Blake",
 "home_8_pos": 5, "home_9_name": "Kelly Shoppach",
 "home_9_pos": 2}
 
datsa = json.dumps(dats)

In [8]:
test_req = requests.post(url,datsa)
print(test_req)

<Response [500]>


In [26]:
#!cd ~

The system cannot find the path specified.
The system cannot find the path specified.


In [29]:
!pwd

/c/Users/dakot/Documents/GitHub/baseball-game-prediction-ds/baseball_notebooks


The system cannot find the path specified.


In [206]:
imps
scrs
# probs

0.5349794238683128

In [207]:
# Pickle the darn thing! 🥒
import pickle 
model_name = 'cat_pickle.pkl'

In [208]:
model_name = 'cat_pickle.pkl'
pickle.dump(kitty, open(model_name, 'wb'))

In [0]:
# explainer = shap.TreeExplainer(kitty) # WHY DOES THIS TAKE SOO LONG?!?!?!?!
# shap_values = explainer.shap_values(X_test)

In [0]:
y_pred = l_model.predict(X_test)

In [0]:
acc = score(y_test,y_pred)

In [0]:
acc

In [0]:
l_model.coef_

In [0]:
print(list(zip(l_model.coef_[0,:], X.columns)))

In [0]:
coef_dict = {}
for coef, feat in zip(l_model.coef_[0,:],X.columns):
    coef_dict[feat] = coef

In [0]:
plt.figure(figsize=(20,20))

plt.bar(*zip(*coef_dict.items()),color='orange')
plt.xticks( X.columns, rotation='vertical', fontsize='20')
plt.grid();
plt.savefig('feature_coefs.png',bbox_inches='tight')

In [0]:
pred_class

In [1]:
!ls

AB-Demo
baseball_predict
Baseball_predict_begin(10-3-19).ipynb
baseball-game-prediction-ds
cat_feature_index.43a53045-2a7bda50-7e2330d6-4aacd9f2.tmp
cat_feature_index.7d127b97-5681c70-cbf40be0-a02adb70.tmp
cat_feature_index.815c90cc-fc8c0611-9d00dd26-cbcdcfe5.tmp
cat_feature_index.e1386f5e-dcf9b259-9fdbfab5-970c22cd.tmp
cat_pickle.pkl
catboost_info
cb_model.json
df_2008
Dpgofast.github.io
DS-Material
DS-Sprint-02-Storytelling-With-Data
DS-Sprint-03-Creating-Professional-Portfolios
DS-Unit-1-Sprint-4-Statistical-Tests-and-Experiments
DS-Unit-2-Applied-Modeling
DS-Unit-2-Linear-Models
DS-Unit-2-Sprint-1-Linear-Algebra
DS-Unit-2-Sprint-2-Linear-Regression
DS-Unit-2-Sprint-2-Regression
DS-Unit-2-Sprint-3-Advanced-Regression
DS-Unit-2-Sprint-3-Classification-Validation
DS-Unit-2-Sprint-4-Model-Validation
DS-Unit-2-Sprint-4-Practicing-Understanding
DS-Unit-3-Sprint-1-Software-Engineering
DS-Unit-3-Sprint-2-SQL-and-Databases
DS-Unit-3-Sprint-3-Big-Data
DS-Unit-3-Sprint-4-Productization-and-Cl